In [4]:
import osmnx as ox
import networkx as nx
from heapq import heappop, heappush
from math import sqrt
import ipyleaflet
from ipyleaflet import Map, Marker, Polyline, LayerGroup, FullScreenControl, basemaps
import ipywidgets as widgets
from IPython.display import display
import time

# Define the area of interest
place = 'Manhattan, New York, USA'
G = ox.graph_from_place(place, network_type='drive')

# Create an ipyleaflet map for interactive pin placement and visualization
m_ipy = Map(center=(40.748817, -73.985428), zoom=13, basemap=basemaps.CartoDB.DarkMatter)
m_ipy.add_control(FullScreenControl())

# Initialize variables to store start and end coordinates
start_coords, end_coords = None, None
markers = LayerGroup(name='Markers')
m_ipy.add_layer(markers)

# Event handler for map clicks
def handle_map_click(**kwargs):
    global start_coords, end_coords
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        if start_coords is None:
            start_coords = coords
            marker = Marker(location=start_coords, draggable=False, title='Start')
            markers.add_layer(marker)
        elif end_coords is None:
            end_coords = coords
            marker = Marker(location=end_coords, draggable=False, title='End')
            markers.add_layer(marker)
        else:
            print("Both start and end coordinates are set")

m_ipy.on_interaction(handle_map_click)

display(m_ipy)

# Define the A* search algorithm and visualization
def euclidean_heuristic(node1, node2):
    lat1, lon1 = G.nodes[node1]['y'], G.nodes[node1]['x']
    lat2, lon2 = G.nodes[node2]['y'], G.nodes[node2]['x']
    return sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

def enhanced_heuristic(node, end, initial_distance):
    current_distance = euclidean_heuristic(node, end)
    penalty_factor = 1000  # Adjust this factor to control the penalty strength
    
    # Penalize paths that increase distance significantly
    if current_distance > initial_distance:
        return current_distance * penalty_factor
    else:
        return current_distance

def a_star_search(G, start, end):
    open_set = []
    initial_distance = euclidean_heuristic(start, end)
    heappush(open_set, (0, start))
    came_from = {}
    g_score = {node: float('inf') for node in G.nodes}
    g_score[start] = 0
    f_score = {node: float('inf') for node in G.nodes}
    f_score[start] = enhanced_heuristic(start, end, initial_distance)

    open_set_hash = {start}
    explored_edges = []

    while open_set:
        current = heappop(open_set)[1]
        open_set_hash.remove(current)

        if current == end:
            path = reconstruct_path(came_from, current)
            yield path, explored_edges, True  # Final result
            return

        for neighbor in G.neighbors(current):
            tentative_g_score = g_score[current] + G[current][neighbor][0]['length']
            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + enhanced_heuristic(neighbor, end, initial_distance)
                if neighbor not in open_set_hash:
                    heappush(open_set, (f_score[neighbor], neighbor))
                    open_set_hash.add(neighbor)
                    explored_edges.append((current, neighbor))
                    yield None, [(current, neighbor)], False  # Intermediate result

    yield None, explored_edges, True  # No path found

def animate_a_star():
    global start_coords, end_coords
    assert start_coords is not None and end_coords is not None, "Set start and end coordinates on the map."
    start_node = ox.distance.nearest_nodes(G, start_coords[1], start_coords[0])
    end_node = ox.distance.nearest_nodes(G, end_coords[1], end_coords[0])

    explored_layer = LayerGroup()
    path_layer = LayerGroup()
    m_ipy.add_layer(explored_layer)
    m_ipy.add_layer(path_layer)

    for path, new_edges, is_final in a_star_search(G, start_node, end_node):
        if not is_final:
            for edge in new_edges:
                line = Polyline(
                    locations=[
                        (G.nodes[edge[0]]['y'], G.nodes[edge[0]]['x']),
                        (G.nodes[edge[1]]['y'], G.nodes[edge[1]]['x'])
                    ],
                    color='blue',
                    opacity=0.5
                )
                explored_layer.add_layer(line)
        else:
            if path:
                path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in path]
                final_polyline = Polyline(locations=path_coords, color='yellow', weight=5)
                path_layer.add_layer(final_polyline)

                route_gdf = ox.utils_graph.route_to_gdf(G, path)
                path_length = route_gdf['length'].sum()
                print(f'Path Length: {path_length:.2f} meters')
            else:
                print("No path found")

        time.sleep(0.05)

def reconstruct_path(came_from, current):
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path.append(current)
    return total_path[::-1]

# Button to trigger A* visualization
button = widgets.Button(description="Run Animated A* Search")
button.on_click(lambda x: animate_a_star())
display(button)


Map(center=[40.748817, -73.985428], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

Button(description='Run Animated A* Search', style=ButtonStyle())

Path Length: 2164.78 meters


C:\Users\aniru\AppData\Local\Temp\ipykernel_3420\4017721264.py:125: FutureWarning: The `route_to_gdf` function has moved to the `routing` module. Calling `utils_graph.route_to_gdf` is deprecated and will be removed in the v2.0.0 release. Call it via `routing.route_to_gdf` instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  route_gdf = ox.utils_graph.route_to_gdf(G, path)
